## 1. Установка необходимых библиотек, включая MediaPipe и OpenCV:

In [42]:
!pip install -q mediapipe opencv-python-headless

In [26]:
import gdown, os, shutil                                                        # Системные инструменты
from zipfile import ZipFile
from tqdm.auto import tqdm                                                      # Модуль для создания прогресс-бара при выполнении циклов
import csv
import mediapipe as mp                                                          # Библиотека MediaPipe
import cv2                                                                      # Библиотека обработки изображений
from google.colab.patches import cv2_imshow

Загружаем архив с Google-диска и распаковываем его

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# shutil.copy('/content/drive/MyDrive/проект/project.zip', '/content/project.zip')

In [ ]:
gdown.download('https://drive.google.com/uc?id=1nlIAlHDh5UI1ZB2S1cIWcFy6FXjhwsJ5', None, quiet=True)

In [39]:
with ZipFile("project.zip", "r") as myzip:
    myzip.extractall()

In [47]:
# shutil.rmtree('/content/project')
# shutil.rmtree('/content/out')

## 3. Получаем список всех видео файлов в папке "project",
а заодно переименовываем их, что бы убрать ошибку в распознавании кириллических символов.

In [40]:
video_files = []
for root, dirs, files in sorted(os.walk('/content/project')):
    n = 1
    for file in files:
        os.rename(os.path.join(root, file), os.path.join(root, f'video_{n}.mp4'))
        video_files.append(os.path.join(root, f'video_{n}.mp4'))
        n += 1
video_files

['/content/project/GedanBaraj_flank_0_right/video_1.mp4',
 '/content/project/GedanBaraj_flank_0_right/video_2.mp4',
 '/content/project/GedanBaraj_flank_0_right/video_3.mp4',
 '/content/project/GedanBaraj_flank_0_right/video_4.mp4',
 '/content/project/GedanBaraj_flank_0_right/video_5.mp4',
 '/content/project/GedanBaraj_flank_0_right/video_6.mp4',
 '/content/project/GedanBaraj_flank_1_body turn by block/video_1.mp4',
 '/content/project/GedanBaraj_flank_1_body turn by block/video_2.mp4',
 '/content/project/GedanBaraj_flank_1_fist is turned wrong/video_1.mp4',
 '/content/project/GedanBaraj_flank_1_fist is turned wrong/video_2.mp4',
 '/content/project/GedanBaraj_flank_1_straight arm swing/video_1.mp4',
 '/content/project/GedanBaraj_flank_1_straight arm swing/video_2.mp4',
 '/content/project/GedanBaraj_front_0_right/video_1.mp4',
 '/content/project/GedanBaraj_front_0_right/video_2.mp4',
 '/content/project/GedanBaraj_front_0_right/video_3.mp4',
 '/content/project/GedanBaraj_front_0_right/vide

Выберем часть файлов

In [44]:
video_files = ['/content/project/UrakenUchi_front_1_strong carry kick/video_1.mp4',
               '/content/project/UrakenUchi_front_1_strong carry kick/video_2.mp4']

## 4. Функция обрабатки видеофайла

In [48]:
mp_pose = mp.solutions.pose                                                     # Создаем объекты для обнаружения скелетных точек на изображении
mp_drawing = mp.solutions.drawing_utils                                         # Создаем объекты для рисование ключевых точек на кадре


def process_video(video_file, size=(1280, 720)):
    dir = video_file.split(sep='/')[-2:]                                        # Разделяем путь к файлу на части: имя файла и имена папок
    if not os.path.exists(f'/content/out/{dir[0]}'):                            # Проверяем, если директории нет, то ...
        os.makedirs(f'/content/out/{dir[0]}')                                   # ... создаем ее
    video_file_out = f'/content/out/{dir[0]}/{dir[1]}'

    with mp_pose.Pose(model_complexity=2,                                       # Используем контекстный менеджер "with" для создания объекта mp_pose.Pose
                      min_detection_confidence=0.5,                             # и задаем параметры для 'pose'
                      min_tracking_confidence=0.5
                      ) as pose:

        cap = cv2.VideoCapture(video_file)                                      # Инициализация чтения видео файла
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))                    # Получаем количество кадров в видео файле
        fps = int(cap.get(cv2.CAP_PROP_FPS))                                    # Получаем частоту кадров

        record = cv2.VideoWriter(                                               # Инициализация записи видео файла
                          video_file_out,                                       # Путь записи видео файла
                          cv2.VideoWriter_fourcc('M','P','4','V'),              # 4-значный код кодека для сжатия кадров
                          fps,                                                  # количество кадров в секунду
                          size,                                                 # размер кадров
                          True)
        n=1
        for i in tqdm(range(frame_count)):                                      # Пробегаем по каждому кадру в видео файле
            ret, image = cap.read()
            if not ret:                                                         # Если кадр не считался, то прекращаем цикл
                print('Кадр не считался, цикл break')
                break
            image = cv2.resize(image, (size))                                   # Приводим размер изображения к размеру видео
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)                      # Переводим картинку в RGB
            results = pose.process(image)                                       # Обрабатываем изображение для обнаружения скелетных точек
            mp_drawing.draw_landmarks(image,                                    # Рисуем ключевые точки на кадре
                                      results.pose_landmarks,
                                      mp_pose.POSE_CONNECTIONS)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)                      # Конверуем кадр обратно в BGR
            image_dir_out = f'/content/out/{dir[0]}/{os.path.splitext(dir[1])[0]}'
            if not os.path.exists(image_dir_out):                               # Проверяем, если директории нет, то ...
                os.makedirs(image_dir_out)                                      # ... создаем ее
            image_file_out = f'{image_dir_out}/file_{n:03d}.jpg'
            cv2.imwrite(image_file_out, image)                                  # сохранем изображения
            n +=1

            record.write(image)                                                 # Записываем текущий кадр
        cap.release()                                                           # Закрываем захват видео
        record.release()                                                        # Закрываем запись

## 5. Обрабатываем видеофайлы из списка и записываем видео с скелетными точками

In [49]:
size = (1280, 720)                                                              # Размер сохраняемого видео

for video_file in tqdm(video_files):
    process_video(video_file, size)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/629 [00:00<?, ?it/s]

  0%|          | 0/654 [00:00<?, ?it/s]